In [1]:
import pandas as pd
import re
from transformers import pipeline

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('/workspaces/compass-academy/sprint6/desafio/dados_raw/movies.csv',
    delimiter='|',
    quotechar='"',        # Specifies the quote character
    escapechar='\\',      # Handles escaped characters
    engine='python'       # Uses the Python parser instead of C
)

In [3]:
df.head()

,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
0,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actress,Miss Geraldine Holbrook (Miss Jerry),Blanche Bayliss,1878,1951,actress,tt0000009
1,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actor,Mr. Hamilton,William Courtenay,1875,1933,actor,"tt0021535,tt0000009,tt0020355,tt0020403"
2,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actor,Chauncey Depew - the Director of the New York ...,Chauncey Depew,1834,1928,"actor,writer","tt1076833,tt0490842,tt0000009,tt4484306"
3,tt0000502,Bohemios,Bohemios,1905,100,N,4.2,14,actor,N,Antonio del Pozo,N,N,actor,tt0000502
4,tt0000502,Bohemios,Bohemios,1905,100,N,4.2,14,actor,N,El Mochuelo,N,N,actor,tt0000502


### Filtro 1: Gêneros Drama / Romance

In [4]:
df1 = df[df['genero'].str.contains(r'^(Drama|Romance|Drama,Romance|Romance,Drama)$', regex=True)]
df1

/tmp/ipykernel_2497/1794277014.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df1 = df[df['genero'].str.contains(r'^(Drama|Romance|Drama,Romance|Romance,Drama)$', regex=True)]


,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
0,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actress,Miss Geraldine Holbrook (Miss Jerry),Blanche Bayliss,1878,1951,actress,tt0000009
1,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actor,Mr. Hamilton,William Courtenay,1875,1933,actor,"tt0021535,tt0000009,tt0020355,tt0020403"
2,tt0000009,Miss Jerry,Miss Jerry,1894,45,Romance,5.3,200,actor,Chauncey Depew - the Director of the New York ...,Chauncey Depew,1834,1928,"actor,writer","tt1076833,tt0490842,tt0000009,tt4484306"
9,tt0000591,The Prodigal Son,L'enfant prodigue,1907,90,Drama,5.1,20,actor,N,Henri Gouget,N,N,actor,"tt3752844,tt0451296,tt0311681,tt0400955"
10,tt0000591,The Prodigal Son,L'enfant prodigue,1907,90,Drama,5.1,20,actress,N,Christiane Mandelys,1873,1957,actress,"tt0757216,tt0208313,tt0757218,tt0000591"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045134,tt9916170,The Rehearsal,O Ensaio,2019,51,Drama,7.0,7,actress,N,Julia Ianina,1983,N,"actress,writer","tt5822138,tt12329148,tt18309812,tt0293007"
1045153,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6,actor,Vikash,Sahil Shah,N,N,actor,"tt10039468,tt7981492,tt9916538"
1045154,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6,actress,Sinta,Lala Karmela,1985,N,actress,"tt6380556,tt9916538,tt8060288,tt6503596"
1045155,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6,actress,Widhi,Cut Mini Theo,1973,N,actress,"tt5935682,tt10495746,tt10329546,tt7541708"


### Filtro 2: Ano 2013 ~ Atual

In [5]:
df1.loc[df1['anoLancamento'].str.contains(r'^[a-zA-Z]+$', regex=True), 'anoLancamento'] = 0
df1['anoLancamento'] = df1['anoLancamento'].astype(int)

/tmp/ipykernel_2497/1949478750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['anoLancamento'] = df1['anoLancamento'].astype(int)


In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 259480 entries, 0 to 1045156
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     259480 non-null  object 
 1   tituloPincipal         259480 non-null  object 
 2   tituloOriginal         259480 non-null  object 
 3   anoLancamento          259480 non-null  int64  
 4   tempoMinutos           259480 non-null  object 
 5   genero                 259480 non-null  object 
 6   notaMedia              259480 non-null  float64
 7   numeroVotos            259480 non-null  int64  
 8   generoArtista          259480 non-null  object 
 9   personagem             259480 non-null  object 
 10  nomeArtista            259480 non-null  object 
 11  anoNascimento          259480 non-null  object 
 12  anoFalecimento         259480 non-null  object 
 13  profissao              257929 non-null  object 
 14  titulosMaisConhecidos  259445 non-null  

In [7]:
df2 = df1[df1['anoLancamento'] >= 2013]
df2

,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
134119,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actress,Fantasma,Claudia Paz,1920,2015,actress,"tt0130375,tt0454087,tt0242237,tt1187044"
134120,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actor,Viudo,Rubén Sotoconil,N,2002,actor,"tt0094084,tt0189469,tt0130375,tt0062336"
134121,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actress,Lola,Shenda Román,1928,N,actress,"tt0064147,tt0067475,tt0063718,tt2128535"
134122,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actor,Silva,Luis Alarcón,1929,N,"actor,special_effects","tt3234226,tt0379108,tt0242198,tt1947646"
152294,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122,Drama,6.7,7455,actor,Brooks Otterlake,Peter Bogdanovich,1939,2022,"actor,director,writer","tt0067328,tt0079843,tt0070510,tt0063671"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045134,tt9916170,The Rehearsal,O Ensaio,2019,51,Drama,7.0,7,actress,N,Julia Ianina,1983,N,"actress,writer","tt5822138,tt12329148,tt18309812,tt0293007"
1045153,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6,actor,Vikash,Sahil Shah,N,N,actor,"tt10039468,tt7981492,tt9916538"
1045154,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6,actress,Sinta,Lala Karmela,1985,N,actress,"tt6380556,tt9916538,tt8060288,tt6503596"
1045155,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama,8.3,6,actress,Widhi,Cut Mini Theo,1973,N,actress,"tt5935682,tt10495746,tt10329546,tt7541708"


### Filtro 3: Filtrar Filmes Distintos

In [8]:
df3 = df2.drop_duplicates(subset=['id'], keep='last')
df3

,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos
134122,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actor,Silva,Luis Alarcón,1929,N,"actor,special_effects","tt3234226,tt0379108,tt0242198,tt1947646"
152297,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122,Drama,6.7,7455,actress,The Actress,Oja Kodar,1941,N,"actress,writer,director","tt0108506,tt0239070,tt0209994,tt0069049"
266695,tt0120589,A Dangerous Practice,A Dangerous Practice,2022,108,Drama,7.0,135,actor,Cícero,Cássio Gabus Mendes,1961,N,"actor,producer","tt0149517,tt0478978,tt1587828,tt0840797"
317301,tt0173042,Premaloka,Premaloka,2021,154,"Drama,Romance",7.9,518,actor,Ravi,V. Ravichandran,1961,N,"actor,director,writer","tt0173189,tt0173042,tt0308793,tt0470192"
339910,tt0192528,Heaven & Hell,Reverse Heaven,2018,104,Drama,3.9,118,actor,Demon in Heaven,Tim Dax,N,N,"actor,producer,director","tt15203302,tt3796746,tt4997978,tt5225574"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045093,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130,Drama,8.0,378,actor,N,Anoop Chandran,N,N,"actor,producer","tt1028558,tt0867145,tt2429086,tt9244322"
1045101,tt9913660,No Apology,No Apology,2019,102,Drama,5.7,66,actress,Young Robin,Kiara Corkett,N,N,actress,"tt6588762,tt5950436,tt9913660,tt6056808"
1045121,tt9914942,Life Without Sara Amat,La vida sense la Sara Amat,2019,74,Drama,6.7,164,actress,Sara,Maria Morera,N,N,actress,"tt10589738,tt9914942,tt14604096,tt14596388"
1045134,tt9916170,The Rehearsal,O Ensaio,2019,51,Drama,7.0,7,actress,N,Julia Ianina,1983,N,"actress,writer","tt5822138,tt12329148,tt18309812,tt0293007"


### Filtro 4: Remoção de Títulos Originais em Inglês

In [9]:
lang_identifier = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

def is_english(text):
    result = lang_identifier(text)
    return result[0]['label'] == 'en'

df3['Is_English'] = df3['tituloOriginal'].apply(is_english)
df3

Device set to use cpu
/tmp/ipykernel_2497/3497210258.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Is_English'] = df3['tituloOriginal'].apply(is_english)


,id,tituloPincipal,tituloOriginal,anoLancamento,tempoMinutos,genero,notaMedia,numeroVotos,generoArtista,personagem,nomeArtista,anoNascimento,anoFalecimento,profissao,titulosMaisConhecidos,Is_English
134122,tt0062336,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,2020,70,Drama,6.4,163,actor,Silva,Luis Alarcón,1929,N,"actor,special_effects","tt3234226,tt0379108,tt0242198,tt1947646",False
152297,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122,Drama,6.7,7455,actress,The Actress,Oja Kodar,1941,N,"actress,writer,director","tt0108506,tt0239070,tt0209994,tt0069049",True
266695,tt0120589,A Dangerous Practice,A Dangerous Practice,2022,108,Drama,7.0,135,actor,Cícero,Cássio Gabus Mendes,1961,N,"actor,producer","tt0149517,tt0478978,tt1587828,tt0840797",True
317301,tt0173042,Premaloka,Premaloka,2021,154,"Drama,Romance",7.9,518,actor,Ravi,V. Ravichandran,1961,N,"actor,director,writer","tt0173189,tt0173042,tt0308793,tt0470192",False
339910,tt0192528,Heaven & Hell,Reverse Heaven,2018,104,Drama,3.9,118,actor,Demon in Heaven,Tim Dax,N,N,"actor,producer,director","tt15203302,tt3796746,tt4997978,tt5225574",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045093,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,130,Drama,8.0,378,actor,N,Anoop Chandran,N,N,"actor,producer","tt1028558,tt0867145,tt2429086,tt9244322",False
1045101,tt9913660,No Apology,No Apology,2019,102,Drama,5.7,66,actress,Young Robin,Kiara Corkett,N,N,actress,"tt6588762,tt5950436,tt9913660,tt6056808",True
1045121,tt9914942,Life Without Sara Amat,La vida sense la Sara Amat,2019,74,Drama,6.7,164,actress,Sara,Maria Morera,N,N,actress,"tt10589738,tt9914942,tt14604096,tt14596388",False
1045134,tt9916170,The Rehearsal,O Ensaio,2019,51,Drama,7.0,7,actress,N,Julia Ianina,1983,N,"actress,writer","tt5822138,tt12329148,tt18309812,tt0293007",False


In [11]:
df3.to_csv(
    './filmes_attr_is_english.csv',
    sep=',',
    header=True,
    index=False
)